# Gradio UI
A simple Gradio UI where a database that holds the reciept json data is saved and queried on using a simple LLM chat UI.

In [1]:
# setup

import os
import base64
import json
import time
import re
from dotenv import load_dotenv
from openai import OpenAI
import sqlite3

load_dotenv()

True

In [2]:
# 1. Path to JSON export from your main pipeline
JSON_PATH = "receipt_output.json"

# 2. Connect to (or create) a local SQLite database file
DB_PATH = "pantry.db"
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# 3. Create a table for receipt items (drop if exists for fresh start)
cursor.execute("""
DROP TABLE IF EXISTS items;
""")
cursor.execute("""
CREATE TABLE items (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    item TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    expiration DATE,
    category TEXT,
    location TEXT,
    cost REAL
);
""")
conn.commit()

# 4. Read JSON and bulk-insert into the database
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

rows = [
    (
        d["item"],
        d.get("quantity", 1),
        None if d.get("expiration") == "N/A" else d["expiration"],
        d.get("category"),
        d.get("location"),
        None if d.get("cost") == "N/A" else d["cost"],
    )
    for d in data
]

cursor.executemany("""
INSERT INTO items (item, quantity, expiration, category, location, cost)
VALUES (?, ?, ?, ?, ?, ?);
""", rows)
conn.commit()

print(f"Loaded {len(rows)} rows into {DB_PATH}.")

Loaded 17 rows into pantry.db.


In [3]:
def query_pantry_db(sql_query: str) -> list:
    """Run a SQL query against the local SQLite DB and return rows as dicts."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute(sql_query)
    results = [dict(row) for row in cur.fetchall()]
    conn.close()
    return results

def ask_pantry(question: str) -> str:
    # 1) Spin up the same OpenAI client you used elsewhere
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # 2) Pull in your pantry rows
    items = query_pantry_db("SELECT * FROM items;")

    # 3) Build the prompt
    prompt = (
        "You are an expert kitchen assistant with access to my pantry database.\n"
        "Here is the current data:\n"
        f"{json.dumps(items, indent=2)}\n\n"
        f"User question: {question}\n"
        "Answer concisely based on the data above."
    )

    # 4) Call the same chat API surface you used before
    chat = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.0,
        max_tokens=300,
        messages=[{"role": "user", "content": prompt}]
    )

    # 5) Return the assistant’s text
    return chat.choices[0].message.content.strip()


In [4]:
# Cell #3: simple Gradio app to ask questions of your digital pantry
import gradio as gr

def gradio_interface(user_question):
    return ask_pantry(user_question)

css = """
/* optional: center content */
.gradio-container { max-width: 800px; margin: auto; }
"""

app = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=2, placeholder="What do I have in my pantry?"),
    outputs="text",
    title="🍎 Pang: Digital Pantry Assistant",
    description="Ask questions like 'What fruit do I have?' or 'List items expiring soon.'",
    css=css,
)

if __name__ == "__main__":
    # bind to 0.0.0.0 so your browser (and cloud IDE) can see it,
    # and create a public link in case localhost is blocked.
    app.launch(
        server_name="0.0.0.0",
        server_port=7861,
        share=True,       # <-- spins up a public URL
        inbrowser=True    # <-- tries to auto-open in your default browser
    )


* Running on local URL:  http://0.0.0.0:7861
* Running on public URL: https://e7e5568587cf392d62.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
